<a href="https://www.kaggle.com/code/sk0903273/used-car-price-prediction?scriptVersionId=196417042" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv("/kaggle/input/playground-series-s4e9/train.csv")
test_df=pd.read_csv("/kaggle/input/playground-series-s4e9/test.csv")
sample_submission_df=pd.read_csv("/kaggle/input/playground-series-s4e9/sample_submission.csv")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
test_df.isna().sum()

In [ ]:
train_df.isna().sum()

In [ ]:
train_df.info()

In [ ]:
train_df['fuel_type'].value_counts()

In [ ]:
test_df['fuel_type'].value_counts()

In [ ]:
train_df['fuel_type']=train_df['fuel_type'].fillna("Gasoline")
train_df['fuel_type']=train_df['fuel_type'].replace('-',"Gasoline")

test_df['fuel_type']=test_df['fuel_type'].fillna("Gasoline")
test_df['fuel_type']=test_df['fuel_type'].replace('-',"Gasoline")

In [ ]:
train_df['accident'].value_counts(),test_df['accident'].value_counts()

In [ ]:
train_df['accident']=train_df['accident'].fillna("None reported")
test_df['accident']=test_df['accident'].fillna("None reported")

In [ ]:
train_df['clean_title']=train_df['clean_title'].fillna("Yes")
test_df['clean_title']=test_df['clean_title'].fillna("Yes")

In [ ]:
from datetime import datetime
train_df['car_age']=datetime.now().year-train_df['model_year']

In [ ]:
test_df['car_age']=datetime.now().year-test_df['model_year']
train_df.drop('model_year',axis=1,inplace=True)
test_df.drop('model_year',axis=1,inplace=True)

In [ ]:
train_df.drop_duplicates(inplace=True)
test_df.drop_duplicates(inplace=True)

In [ ]:
train_df['HP'] =train_df['engine'].str.split(' ').str[0]
train_df['HP'] = train_df['HP'].str.slice(0,3)
train_df['HP'] = train_df['HP'].replace(r'[a-zA-Z\W]' ,0, regex =True)
train_df['HP'].astype(int)


test_df['HP'] =test_df['engine'].str.split(' ').str[0]
test_df['HP'] = test_df['HP'].str.slice(0,3)
test_df['HP'] = test_df['HP'].replace(r'[a-zA-Z\W]' ,0, regex =True)
test_df['HP'].astype(int)

In [ ]:
train_df.drop('engine',axis=1,inplace=True)
test_df.drop('engine',axis=1,inplace=True)

In [ ]:
categorical_columns=[feature for feature in train_df.columns if train_df[feature].dtype=='O']
numerical_columns=[feature for feature in train_df.columns if train_df[feature].dtype!='O']

In [ ]:
for f in categorical_columns:
    print(f"{f} : {train_df[f].nunique()}")

In [ ]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
categorical_columns = train_df.select_dtypes(include ='object').columns
for column in categorical_columns:
    train_df[column] = train_df[column].astype(str)
    test_df[column] = test_df[column].astype(str)
    test_df[column] = label.fit_transform(test_df[column])
    train_df[column] = label.fit_transform(train_df[column])

In [ ]:
from sklearn.model_selection import train_test_split
x = train_df.drop(columns = 'price')
y = train_df['price']
x_train, x_test, y_train,y_test = train_test_split(x, y,test_size = 0.2, random_state =42)

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

params = {'learning_rate': 0.042271457347806164,
          'max_depth': 6, 
          'subsample': 0.8206635132608892,
          'colsample_bytree': 0.5512265074973758,
          'n_estimators': 293, 
          'min_child_weight': 9, 
          'gamma': 0.38289919165608166, 
          'reg_alpha': 0.06782645735387109, 
          'reg_lambda': 0.0683153554753663}

model = XGBRegressor(**params)

XGB = model.fit(
    x_train,
    y_train,
    eval_set=[(x_test, y_test)],
    eval_metric='rmse', 
)

In [ ]:
y_pred = model.predict(x_test)
rmse_score = mean_squared_error(y_test, y_pred, squared=False) 
print("RMSE Score:", rmse_score)

In [ ]:
test_pred = model.predict(test_df)

res = pd.DataFrame({'id': test_df['id'],'class': test_pred})

In [ ]:
res.to_csv('submission.csv', index=False)